In [1]:
import os
import glob
import numpy as np
import torch, gc
import pandas as pd
#import cudf
#import cupy
import nvtabular as nvt
#from merlin.dag import ColumnSelector
#from merlin.schema import Schema, Tags

ModuleNotFoundError: No module named 'merlin.dag'

In [53]:
df= pd.read_csv('train_and_test_full.csv')

In [54]:
df.head()

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,istest,submission,N,utrip_id_
0,29,2016-07-09,2016-07-11,47054,desktop,1601,Elbonia,Elbonia,29_1,0,0,4,0
1,29,2016-07-11,2016-07-13,34444,desktop,1601,Elbonia,Elbonia,29_1,0,0,4,0
2,29,2016-07-13,2016-07-16,12291,desktop,1601,Elbonia,Elbonia,29_1,0,0,4,0
3,29,2016-07-16,2016-07-18,16386,desktop,8132,Elbonia,Elbonia,29_1,0,0,4,0
4,65,2016-09-26,2016-09-29,36403,desktop,3577,The Devilfire Empire,Cobra Island,65_1,1,0,5,1


In [55]:
trips_df=df[(df['N']>1) & (df['N']<10)]

In [56]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391808 entries, 0 to 1545501
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   user_id         1391808 non-null  int64 
 1   checkin         1391808 non-null  object
 2   checkout        1391808 non-null  object
 3   city_id         1391808 non-null  int64 
 4   device_class    1391808 non-null  object
 5   affiliate_id    1391808 non-null  int64 
 6   booker_country  1391808 non-null  object
 7   hotel_country   1324249 non-null  object
 8   utrip_id        1391808 non-null  object
 9   istest          1391808 non-null  int64 
 10  submission      1391808 non-null  int64 
 11  N               1391808 non-null  int64 
 12  utrip_id_       1391808 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 148.7+ MB


In [57]:
del df
gc.collect()

1928

In [70]:
trips_df.checkin=pd.to_datetime(trips_df.checkin.to_numpy(), infer_datetime_format=True)

In [72]:
trips_df.checkin.dtype

dtype('<M8[ns]')

In [66]:
trips_df.checkout=pd.to_datetime(trips_df.checkout.to_numpy(), infer_datetime_format=True)

In [74]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391808 entries, 0 to 1545501
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   user_id         1391808 non-null  int64 
 1   checkin         1391808 non-null  object
 2   checkout        1391808 non-null  object
 3   city_id         1391808 non-null  int64 
 4   device_class    1391808 non-null  object
 5   affiliate_id    1391808 non-null  int64 
 6   booker_country  1391808 non-null  object
 7   hotel_country   1324249 non-null  object
 8   utrip_id        1391808 non-null  object
 9   istest          1391808 non-null  int64 
 10  submission      1391808 non-null  int64 
 11  N               1391808 non-null  int64 
 12  utrip_id_       1391808 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 148.7+ MB


In [75]:
# Encodes categorical features as contiguous integers
cat_feats = ColumnSelector(['city_id', 'device_class', 'booker_country', 'hotel_country', 'utrip_id']) >> nvt.ops.Categorify(start_index=1)

# ENGINEER MONTH, WEEKDAY, and LENGTH OF STAY FEATURES
#raw['mn'] = raw.checkin.dt.month
#raw['dy1'] = raw.checkin.dt.weekday
#raw['dy2'] = raw.checkout.dt.weekday
#raw['length'] = (raw.checkout - raw.checkin).dt.days

# create time features
checkin = ColumnSelector(['checkin'])
month = (
    checkin >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col.astype(str),infer_datetime_format=True).dt.month) >> 
    nvt.ops.Rename(postfix = '_month')
)
checkin_weekday = (
    checkin >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col.astype(str),infer_datetime_format=True).dt.weekday) >> 
    nvt.ops.Rename(postfix ='_dw')
)

checkout = ColumnSelector(['checkout'])
month1 = (
    checkout >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col.astype(str),infer_datetime_format=True).dt.month) >> 
    nvt.ops.Rename(postfix = '_month')
)
checkout_weekday = (
    checkout >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col.astype(str),infer_datetime_format=True).dt.weekday) >> 
    nvt.ops.Rename(postfix ='_dw')
)


time_features = (month + checkin_weekday + month + checkout_weekday)

features = ColumnSelector(['checkin','checkout']) + cat_feats + time_features

In [11]:
# Return top4 metric
# istest: flag to select if metric should be computed in 0:train, 1:test, -1:both
# pos: select which city to calculate the metric, 0: last, 1: last-1, 2:last-2 , -1: all
# the input `val` dataframe must contains the target `city_id` and the 4 recommendations as: rec0, res1, rec2 and rec3

def top4_metric( val, istest=0, pos=0 , target='city_id'):
    
    if istest>=0:
        val = val.loc[ (val.submission==0) & (val.istest == istest) ]
    else:
        val = val.loc[ (val.submission==0) ]

    if pos >= 0:
        top1 = val.loc[val.icount==pos,target] == val.loc[val.icount==pos,'rec0']
        top2 = val.loc[val.icount==pos,target] == val.loc[val.icount==pos,'rec1']
        top3 = val.loc[val.icount==pos,target] == val.loc[val.icount==pos,'rec2']
        top4 = val.loc[val.icount==pos,target] == val.loc[val.icount==pos,'rec3']
    else:
        top1 = val[target] == val['rec0']
        top2 = val[target] == val['rec1']
        top3 = val[target] == val['rec2']
        top4 = val[target] == val['rec3']
        
    return (top1|top2|top3|top4).mean()    

In [76]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    gc.collect()
    torch.cuda.empty_cache()

    cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()
    
    

In [77]:
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 86% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% | 84% |


In [78]:
dataset = nvt.Dataset(trips_df)
workflow = nvt.Workflow(features)
# Learns features statistics necessary of the preprocessing workflow
workflow.fit(dataset)
# Apply the preprocessing workflow in the dataset and converts the resulting Dask cudf dataframe to a cudf dataframe
%time
sessions_gdf = workflow.transform(dataset).compute()
%time

Failed to transform operator <nvtabular.ops.categorify.Categorify object at 0x7fdaa76e1670>
Traceback (most recent call last):
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/site-packages/nvtabular/ops/categorify.py", line 463, in transform
    encoded = _encode(
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/site-packages/nvtabular/ops/categorify.py", line 1372, in _encode
    labels = codes.merge(
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/site-packages/cudf/core/indexed_frame.py", line 1901, in sort_values
    out = self._gather(
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/site-packages/cudf/core/indexed_frame.py", line 1506, in _gather
    libcudf.copying.gather(
  File "copying.pyx", line 185, in cudf._lib.copying.gather
MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/junaid_id/anaconda3/envs/rapids-22.08/include/rmm/mr/device/cuda_memory_resource.hpp

The above exception was the di

RuntimeError: Failed to categorical encode column booker_country

In [41]:
sessions_gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 69746 entries, 0 to 69745
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   city_id         69746 non-null  int64
 1   device_class    69746 non-null  int64
 2   booker_country  69746 non-null  int64
 3   hotel_country   69746 non-null  int64
 4   utrip_id        69746 non-null  int64
 5   checkin         69746 non-null  datetime64[ns]
 6   checkout        69746 non-null  datetime64[ns]
 7   checkout_month  69746 non-null  int16
 8   checkout_dw     69746 non-null  int16
dtypes: datetime64[ns](2), int16(2), int64(5)
memory usage: 4.0 MB


In [15]:
sessions_gdf

,city_id,device_class,booker_country,hotel_country,utrip_id,checkin,checkout,checkout_month,checkout_dw
0,846,2,4,8,5501,2016-07-09,2016-07-11,7,0
1,1525,2,4,8,5501,2016-07-11,2016-07-13,7,2
2,2537,2,4,8,5501,2016-07-13,2016-07-16,7,5
3,4795,2,4,8,5501,2016-07-16,2016-07-18,7,0
4,13,2,3,6,3843,2017-02-03,2017-02-05,2,6
...,...,...,...,...,...,...,...,...,...
42428,9,3,3,3,3055,2016-11-18,2016-11-19,11,5
42429,84,2,2,22,9307,2016-07-18,2016-07-22,7,4
42430,2504,2,2,22,9307,2016-07-22,2016-07-27,7,2
42431,2471,2,2,22,9307,2016-07-27,2016-07-31,7,6


In [11]:
# Define Groupby Operator
groupby_features = ColumnSelector(['utrip_id','city_id','checkin','checkout','booker_country','hotel_country']) >> nvt.ops.Groupby(
    groupby_cols=["utrip_id"], 
    sort_cols=["checkin"],
    aggs={
        'city_id': ["list","count"],
        'checkin': ["first"],
        'checkout': ["last"],
        'booker_country': ["first"],
        'hotel_country': ["list"],
        'checkout_month': ['list'],
        'checkout_dw': ['list']
        },
    name_sep="-") >> nvt.ops.AddMetadata(tags=[Tags.CATEGORICAL])



In [12]:
dataset = nvt.Dataset(sessions_gdf)
workflow = nvt.Workflow(groupby_features)
# Learns features statistics necessary of the preprocessing workflow
%time
workflow.fit(dataset)


CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.48 µs


In [13]:
%time
# Apply the preprocessing workflow in the dataset and converts the resulting Dask cudf dataframe to a cudf dataframe
trips_gdf = workflow.transform(dataset).compute()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.05 µs


In [14]:
trips_gdf

,utrip_id,city_id-list,checkin-first,checkout-last,booker_country-first,hotel_country-list,city_id-count
0,2,"[2341, 1824, 1560, 3075, 2]",2016-09-08,2016-09-13,2,"[5, 5, 5, 5, 1]",5
1,3,"[5142, 997, 1732, 1732, 1732]",2016-04-12,2016-04-28,2,"[2, 2, 2, 2, 2]",5
2,4,"[275, 234, 31, 43, 121]",2016-08-17,2016-08-25,5,"[40, 40, 16, 16, 16]",5
3,5,"[8058, 3898, 1313, 51, 2]",2016-04-21,2016-04-27,3,"[57, 57, 57, 8, 1]",5
4,6,"[3653, 8094, 499, 1191, 2872]",2016-07-31,2016-08-06,3,"[9, 9, 9, 9, 9]",5
...,...,...,...,...,...,...,...
9797,9799,"[15, 3088]",2016-04-25,2016-05-01,3,"[13, 13]",2
9798,9800,"[1271, 309]",2016-06-06,2016-06-09,4,"[27, 27]",2
9799,9801,"[98, 7]",2016-04-25,2016-04-27,3,"[3, 3]",2
9800,9802,"[7266, 619]",2016-07-20,2016-07-23,4,"[8, 8]",2


In [32]:
trips_gdf.to_csv("trips.csv")

NotImplementedError: Writing to csv format is not yet supported with list columns.

In [21]:
dataset = nvt.Dataset(trips_gdf)
workflow = nvt.Workflow(final_features)
# Learns features statistics necessary of the preprocessing workflow
%time
workflow.fit(dataset)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs


ValueError: Missing columns ['duration'] found in operatorSelectionOp during compute_selector.

In [25]:
final_features = trips_gdf.columns.to_list()

In [26]:
trips_gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 9802 entries, 0 to 9801
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   utrip_id              9802 non-null   int64
 1   city_id-list          9802 non-null   list
 2   checkin-first         9802 non-null   datetime64[ns]
 3   checkout-last         9802 non-null   datetime64[ns]
 4   booker_country-first  9802 non-null   int64
 5   hotel_country-list    9802 non-null   list
 6   city_id-count         9802 non-null   int32
 7   duration              9802 non-null   float64
dtypes: datetime64[ns](2), float64(1), int32(1), int64(2), list(2)
memory usage: 1.1 MB


In [27]:
x=ColumnSelector(['duration'])    
duration_norm = ( x >> 
    nvt.ops.Normalize() >> 
    nvt.ops.Rename(postfix = '_norm')
)

In [28]:
dataset = nvt.Dataset(trips_gdf)
workflow = nvt.Workflow(final_features)
# Learns features statistics necessary of the preprocessing workflow
%time
workflow.fit(dataset)

AttributeError: 'list' object has no attribute 'parents_with_dependencies'

In [29]:
final_features =final_features + duration_norm

In [24]:
trips_gdf['duration']=(trips_gdf['checkout-last']-trips_gdf['checkin-first'])/np.timedelta64(1,'D')


In [30]:
%time
# Apply the preprocessing workflow in the dataset and converts the resulting Dask cudf dataframe to a cudf dataframe
trips_gdf = workflow.transform(dataset).compute()

Failed to transform operator <nvtabular.ops.normalize.Normalize object at 0x7fdca014d9a0>
Traceback (most recent call last):
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/site-packages/merlin/dag/executors.py", line 170, in _transform_data
    output_data = node.op.transform(selection, input_data)
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/home/junaid_id/anaconda3/envs/rapids-22.08/lib/python3.9/site-packages/nvtabular/ops/normalize.py", line 79, in transform
    if self.stds[name] > 0:
KeyError: 'duration'


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


KeyError: 'duration'

In [31]:
trips_gdf

,utrip_id,city_id-list,checkin-first,checkout-last,booker_country-first,hotel_country-list,city_id-count,duration
0,2,"[2341, 1824, 1560, 3075, 2]",2016-09-08,2016-09-13,2,"[5, 5, 5, 5, 1]",5,5.0
1,3,"[5142, 997, 1732, 1732, 1732]",2016-04-12,2016-04-28,2,"[2, 2, 2, 2, 2]",5,16.0
2,4,"[275, 234, 31, 43, 121]",2016-08-17,2016-08-25,5,"[40, 40, 16, 16, 16]",5,8.0
3,5,"[8058, 3898, 1313, 51, 2]",2016-04-21,2016-04-27,3,"[57, 57, 57, 8, 1]",5,6.0
4,6,"[3653, 8094, 499, 1191, 2872]",2016-07-31,2016-08-06,3,"[9, 9, 9, 9, 9]",5,6.0
...,...,...,...,...,...,...,...,...
9797,9799,"[15, 3088]",2016-04-25,2016-05-01,3,"[13, 13]",2,6.0
9798,9800,"[1271, 309]",2016-06-06,2016-06-09,4,"[27, 27]",2,3.0
9799,9801,"[98, 7]",2016-04-25,2016-04-27,3,"[3, 3]",2,2.0
9800,9802,"[7266, 619]",2016-07-20,2016-07-23,4,"[8, 8]",2,3.0


In [187]:
workflow.save('workflow_etl')

In [29]:
trips_gdf.to_parquet('output.parquet')


In [33]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged
0,duration_norm,(Tags.CONTINUOUS),float64,False,False
1,utrip_id,(),int64,False,False
2,city_id-list,(),int64,True,True
3,checkin-first,(),datetime64[ns],False,False
4,checkout-last,(),datetime64[ns],False,False
5,booker_country-first,(),int64,False,False
6,hotel_country-list,(),int64,True,True
7,city_id-count,(),int32,False,False
8,duration,(),float64,False,False


In [34]:
workflow.fit_transform(dataset).to_parquet("processed_nvt")

AttributeError: 'Dataset' object has no attribute 'to_csv'

In [27]:
#del sessions_gdf
gc.collect()

7483